# Пример 1

In [1]:
# Работа с матрицами
import numpy as np
# Линейное программирование
from scipy.optimize import linprog

Двое играют на деньги, одновременно называя одно из чисел 1 или 2, 
и потом считая сумму $S$. Если $S$ четная, то первый выигрывает у второго $S$ долларов,
если $S$ нечетная, то второй выигрывает у первого $S$ долларов. 

Матрица игры
$$
	A=\begin{pmatrix} 2 & -3 \\ -3 & 4\end{pmatrix}
$$

In [2]:
# Pay-off matrix
A = np.array([ [2, -3], [-3, 4] ])
print(A)

[[ 2 -3]
 [-3  4]]


## Верхняя и нижняя цена игры

\begin{align*}
    \overline{\nu}&=\min_{j}\max_i a_{ij} & \underline{\nu}&=\max_i\min_j a_{ij}
\end{align*}

In [3]:
# Нижняя цена игры
max( A.min(axis=1) )

-3

In [4]:
# Верхняя цена игры
min( A.max(axis=0) )

2

Они не равны $\Longrightarrow$ нет равновесия Нэша в чистых стратегиях

## Равновесие Нэша в чистых стратегиях

In [5]:
Nash_eq = np.full(A.shape, 'No ')

for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        if (A[i,j]==max(A[:,j])) & (A[i,j]==min(A[i,:])):
            Nash_eq[i,j]='Yes'

print(Nash_eq)

[['No ' 'No ']
 ['No ' 'No ']]


## Ожидаемый выигрыш
Предположим, что игроки следуют смешанным стратегиям 
\begin{align*}
    P^\top&=\begin{pmatrix} 0.3 & 0.7\end{pmatrix} & 
    Q^\top&=\begin{pmatrix} 0.25 & 0.75\end{pmatrix}.
\end{align*}
Ожидаемый выигрыш каждого из игроков:

- Ожидаемый выигрыш первого $EU_A(P,Q)=P^\top AQ$
- Ожидаемый выигрыш второго $EU_B(P,Q)=-EU_A(P,Q)$

In [6]:
P = np.array([0.3, 0.7])
Q = np.array([0.25, 0.75])
# Ожидаемый выигрыш. @ - матричное умножение в numpy
P.T@A@Q

1.0499999999999998

## Равновесие Нэша в смешанных стратегиях
Библиотека `scipy.optimize.linprog` [решает](https://docs.scipy.org/doc/scipy/tutorial/optimize.html#linear-programming-linprog) задачу линейного программирования в виде
\begin{gather*}
    \min c^\top x \\ s.t.\left\{\begin{aligned} 
    A_{ub}x &\leq b_{ub} \\ A_{eq}x &= b_{eq} \\ l\leq x&\leq u
    \end{aligned}\right.
\end{gather*}
**Важно**: по умолчанию в `linprog` используются ограничения неотрицательности переменных

Матрица $A$ имеет отрицательные элементы $\Longrightarrow$ для задачи оптимизации все элементы матрицы $A$ нужно сдвинуть на одну и ту же величину, чтобы они стали положительными.
Например, на 4 (переменная `shift`).

### Первый игрок
Первый игрок решает задачу оптимизации
\begin{gather*}
    \begin{gathered} \min 1^\top x\\ s.t. \left\{\begin{aligned}
    (A+4)^\top x &\geq 1 \\ x&\geq0
    \end{aligned}\right.
    \end{gathered} \Longrightarrow 
    \begin{gathered} \min c^\top x\\ s.t. \left\{\begin{aligned}
    A_{ub} x &\leq b_{ub} \\ 0&\leq x
    \end{aligned}\right.
    \end{gathered}
\end{gather*}
где $c^\top=\begin{pmatrix} 1 & 1 \end{pmatrix}$, $b^\top_{ub}=\begin{pmatrix} -1 & -1 \end{pmatrix}$, $A_{ub}=-A^\top-4$

Пусть в задаче на $\min$: $\hat{x}$ - оптимальное решение, $\hat{f}$ - оптимальное значение целевой функции. Тогда

- Цена игры (ожидаемый выигрыш первого) $1/\hat{f}-4$
- равновесная стратегия первого игрока $P^*=\hat{x}/\hat{f}$

In [ ]:
c = np.array([1,1])
b = np.array([-1, -1])

shift = 4

res = linprog(c, A_ub=-A.T-shift, b_ub=b)

# цена игры, ожидаемы выигрыш первого
value1 = 1/res.fun-shift
# равновесная стратегия
P_star = res.x/res.fun

P_star, value1

(array([0.58333333, 0.41666667]), -0.08333333333333304)

### Второй игрок
Второй игрок решает задачу оптимизации
\begin{gather*}
    \begin{gathered} \max 1^\top y\\ s.t. \left\{\begin{aligned}
    (A+4) y &\leq 1 \\ y&\geq0
    \end{aligned}\right.
    \end{gathered} \Longrightarrow 
    \begin{gathered} \min c^\top y\\ s.t. \left\{\begin{aligned}
    A_{ub} y &\leq b_{ub} \\ 0&\leq y
    \end{aligned}\right.
    \end{gathered}
\end{gather*}
где $c^\top=\begin{pmatrix} -1 & -1 \end{pmatrix}$, $b^\top_{ub}=\begin{pmatrix} 1 & 1 \end{pmatrix}$, $A_{ub}=A+4$

Пусть в задаче на $\min$: $\hat{x}$ - оптимальное решение, $\hat{f}$ - оптимальное значение целевой функции. Тогда

- ожидаемый выигрыш второго $1/\hat{f}+4$
- равновесная стратегия первого игрока $Q^*=-\hat{x}/\hat{f}$

In [ ]:
c = np.array([-1, -1])
b = np.array([1, 1])

shift = 4

res = linprog(c, A_ub=A+shift, b_ub=b)

# ожидаемы выигрыш второго = -value1
value2 = 1/res.fun+shift
# равновесная стратегия
Q_star = -res.x/res.fun

Q_star, value2

(array([0.58333333, 0.41666667]), 0.08333333333333393)